In [1]:
import pandas as pd
import numpy as np

In [2]:
path = '/home/kai/data/kaggle/talkingdata/data/'
train = pd.read_csv(path + 'train_cleaned_final.csv')
test = pd.read_csv(path + 'test_cleaned_final.csv')


In [3]:
load_path = '/home/kai/data/kaggle/talkingdata/wl/data/equalhour/'
file_format = '{}_features_matrixFactv1.csv'
day_list = ['day7', 'day8', 'day9']
df_dict = {}
for file in ['day7', 'day8', 'day9', 'test']: 
    df_dict[file] = pd.read_csv(load_path+file_format.format(file))
    print(file_format.format(file))

day7_features_matrixFactv1.csv
day8_features_matrixFactv1.csv
day9_features_matrixFactv1.csv
test_features_matrixFactv1.csv


# Get Compensate Group

## All columns

In [4]:
def intersec_category(df_train_all, df_test_all, df, col_list):
    missing = {}
    for col in col_list:
        print('processing: {}'.format(col))
        train_index = set(df_train_all[col].value_counts().index)
        test_index = set(df_test_all[col].value_counts().index)                 
        inter_index = list(train_index.intersection(test_index))
        print('whole train index length is: {}'.format(len(train_index)))
        print('test index length is: {}'.format(len(test_index)))
        print('intersection index length is: {}'.format(len(inter_index)))
        miss_value = set(inter_index) - set(df[col].value_counts().index)
        print('missing length: {}'.format(len(miss_value)))
        missing[col] = list(miss_value)
        print('---')
    return missing
col_list = [ 'app', 'device', 'os', 'channel', 'ip']
tmp = intersec_category(train.copy(), test.copy(), pd.concat([df_dict['day7'],df_dict['day8'],df_dict['day9']]), col_list)

processing: app
whole train index length is: 706
test index length is: 417
intersection index length is: 393
missing length: 21
---
processing: device
whole train index length is: 3475
test index length is: 1985
intersection index length is: 1661
missing length: 224
---
processing: os
whole train index length is: 800
test index length is: 395
intersection index length is: 339
missing length: 39
---
processing: channel
whole train index length is: 202
test index length is: 178
intersection index length is: 178
missing length: 0
---
processing: ip
whole train index length is: 277396
test index length is: 93936
intersection index length is: 38164


KeyError: 'ip'

In [ ]:
history_index = set(train.index) - set(df_dict['day7'].index) -set(df_dict['day8'].index) -set(df_dict['day9'].index)

# evaluate

In [14]:
def intersec_category(df_train_all, df_test_all, df, col_list):
    for col in col_list:
        print('processing: {}'.format(col))
        train_index = set(df_train_all[col].value_counts().index)
        test_index = set(df_test_all[col].value_counts().index)                 
        inter_index = list(train_index.intersection(test_index))
        print('train index length is: {}'.format(len(train_index)))
        print('test index length is: {}'.format(len(test_index)))
        print('intersection index length is: {}'.format(len(inter_index)))
        index_map = pd.Series(inter_index, index=inter_index)
        df.loc[:,col] = df[col].map(index_map).fillna(-1)
        print('---')
    return df
col_list = [ 'app', 'device', 'os', 'channel']
tmp = intersec_category(pd.concat([df_dict['day7'],df_dict['day8'],df_dict['day9']]), test.copy(), test.copy(),col_list)
    

processing: app
train index length is: 556
test index length is: 417
intersection index length is: 372
---
processing: device
train index length is: 2438
test index length is: 1985
intersection index length is: 1437
---
processing: os
train index length is: 492
test index length is: 395
intersection index length is: 300
---
processing: channel
train index length is: 193
test index length is: 178
intersection index length is: 178
---


In [3]:
### channel app os

In [4]:
day = train['day'].value_counts()

In [10]:
group = train.groupby(['day'])

In [54]:
# val - front
total_length = len(train)
val_length = 1600 * 10000
train_length = 5900 * 10000

valset = {}
trainset = {}
lgb_val = train.iloc[total_length-(val_length+train_length): total_length-train_length]
lgb_train = train.iloc[total_length-train_length: ]

for col in ['channel', 'app', 'os', 'device', 'ip']:
    valset[col] = set(lgb_val[col].value_counts().index.values)
#     valset[col] = set(train.loc[total_length-75000000: total_length-52500000,col].value_counts().index.values)
    trainset[col] = set(lgb_train[col].value_counts().index.values)

In [38]:
# val - rear
total_length = len(train)
val_length = 2000 * 10000
train_length = 5500 * 10000

valset = {}
trainset = {}
lgb_val = train.iloc[total_length-val_length:]
lgb_train = train.iloc[total_length-(val_length+train_length): total_length-val_length]

for col in ['channel', 'app', 'os', 'device']:
    valset[col] = set(lgb_val[col].value_counts().index.values)
#     valset[col] = set(train.loc[total_length-75000000: total_length-52500000,col].value_counts().index.values)
    trainset[col] = set(lgb_train[col].value_counts().index.values)

In [39]:
#### val
total_length = len(train)
val_length = 2150 * 10000
train_length = 5500 * 10000
val_front = 1
val_front_length = int(val_length * val_front)
val_rear_length = val_length - val_front_length
valset = {}
trainset = {}

lgb_val_front = train.iloc[total_length-(val_length+train_length): total_length-(val_length+train_length) + val_front_length]
print(len(lgb_val_front))
lgb_val_rear = train.iloc[total_length-(val_rear_length):]
print(len(lgb_val_rear))
lgb_val = pd.concat([lgb_val_front,lgb_val_rear])
lgb_train = train.iloc[total_length-(val_length+train_length) + val_front_length: total_length-(val_rear_length)]

print(len(lgb_val))
print(len(lgb_train))

for col in ['channel', 'app', 'os', 'device']:
    trainset[col] = set(lgb_train[col].value_counts().index.values)
    valset[col] = set(lgb_val[col].value_counts().index.values)

21500000
0
21500000
55000000


# In val not in train

app - 5%  
device - 16.3%  
os - 14.1%  
channel - 0.0%  

# evaluate IP
ip intersection is 38164
train

In [20]:
print(len(train.ip.value_counts()))
print(len(test.ip.value_counts()))

277396
93936


In [5]:
ip_inter = set(train.ip.value_counts().index).intersection(set(test.ip.value_counts().index))
ip_array = np.array(ip_inter)
np.save('/home/kai/data/kaggle/talkingdata/wl/data/ip/intersection_ip_values.npy', ip_array)

In [25]:
ip_inter_values = np.array(list(ip_inter))
mapping = pd.Series(ip_inter_values, index=ip_inter_values)

In [30]:
train_inter = train.ip.map(mapping).fillna(-1).astype('int64')
test_inter = test.ip.map(mapping).fillna(-1).astype('int64')

In [34]:
print(len(train_inter.value_counts()))
print(len(test_inter.value_counts()))

38165
38165


In [35]:
train_inter = train_inter[train_inter>=0]
test_inter = test_inter[test_inter>=0]

In [36]:
print(len(train_inter))
print(len(test_inter))

147863120
17263366


In [39]:
print(len(train_inter.value_counts()))

38164


In [37]:

train_inter_index = train_inter.index.values
np.save('/home/kai/data/kaggle/talkingdata/wl/data/ip/train_ip_inter_index.npy', train_inter_index)

In [38]:

test_inter_index = test_inter.index.values
np.save('/home/kai/data/kaggle/talkingdata/wl/data/ip/test_ip_inter_index.npy', test_inter_index)